# Lab 10

**Submission requirements**

1. **HTML Version:** Submit an HTML version of your notebook. Ensure all code outputs are visible. (Export via VS Code: Notebook > Export > HTML).
2. **Colab Link:** Provide a link to your notebook hosted on Google Colab for interactive review.

## Setup

Uncomment and run the following cell to install the required packages.

In [ ]:
# %pip install duckdb leafmap lonboard

In [1]:
import duckdb
import leafmap

## Question 1

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table. 

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [ ]:
leafmap.download_from_url('https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip')

In [2]:
con= duckdb.connect()
con.install_extension('spatial')
con.load_extension('spatial')

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS nyc_census_blocks AS 
        SELECT * FROM ST_Read('nyc_census_blocks.shp')
        """
)

In [3]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS nyc_neighborhoods AS 
        SELECT * FROM ST_Read('nyc_neighborhoods.shp')
        """
)

In [4]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS nyc_streets AS 
        SELECT * FROM ST_Read('nyc_streets.shp')
        """
)

In [5]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS nyc_subway_stations AS 
        SELECT * FROM ST_Read('nyc_subway_stations.shp')
        """
)

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS nyc_homicides AS 
        SELECT * FROM ST_Read('nyc_homicides.shp')
        """
)

## Question 2

Visualize the `nyc_subway_stations` and `nyc_streets` datasets on the same map using leafmap and lonboard.

In [6]:
subway_stations_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_subway_stations").df()
subway_stations_df.head()


,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (583521.854408956 4507077.862599085)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (583324.4866324601 4506805.373160211)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (583304.1823994748 4506069.654048115)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (590250.10594797 4518558.019924332)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (590454.7399891173 4519145.719617855)


In [7]:
subway_stations_gdf = leafmap.df_to_gdf(subway_stations_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
subway_stations_gdf.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (-74.01122 40.71038)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01359 40.70795)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01393 40.70132)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (-73.92992 40.81308)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (-73.92741 40.81835)


In [16]:
leafmap.view_vector(subway_stations_gdf, get_radius=50, get_fill_color='blue')

Map(layers=[ScatterplotLayer(get_fill_color=(0, 0, 255, 255), get_radius=50.0, table=pyarrow.Table
OBJECTID: u…

In [12]:
streets_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_streets").df()
streets_df.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,LINESTRING (586785.4767897038 4492901.00145547...
1,2,None,None,footway,LINESTRING (586645.0073625665 4504977.75036058...
2,3,Avenue O,None,residential,LINESTRING (586750.3019977848 4496109.72213903...
3,4,Walsh Ct,None,residential,"LINESTRING (586728.695515043 4497971.05313857,..."
4,5,None,None,motorway_link,LINESTRING (586587.0531467082 4510088.25040298...


In [13]:
streets_gdf = leafmap.df_to_gdf(streets_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
streets_gdf.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,"LINESTRING (-73.97454 40.58235, -73.97320 40.5..."
1,2,None,None,footway,"LINESTRING (-73.97454 40.69114, -73.97431 40.6..."
2,3,Avenue O,None,residential,"LINESTRING (-73.97452 40.61126, -73.97348 40.6..."
3,4,Walsh Ct,None,residential,"LINESTRING (-73.97451 40.62802, -73.97265 40.6..."
4,5,None,None,motorway_link,"LINESTRING (-73.97452 40.73718, -73.97386 40.7..."


In [14]:
leafmap.view_vector(streets_gdf, get_color='red')

Map(layers=[PathLayer(get_color=(255, 0, 0, 255), get_width=5.0, table=pyarrow.Table
ID: uint16
NAME: string
O…

In [19]:
import leafmap.deckgl as leafmap

In [20]:
m = leafmap.Map()
m.add_vector(subway_stations_gdf, get_radius=50, get_fill_color='blue')
m.add_vector(streets_gdf, get_color='red')
m

Map(layers=[ScatterplotLayer(get_fill_color=(0, 0, 255, 255), get_radius=50.0, table=pyarrow.Table
OBJECTID: u…

## Question 3

Find out what neighborhood the `BLUE` subway stations are in.

In [ ]:
# Add your code here

## Question 4

Find out what streets are within 200 meters of the `BLUE` subway stations.

In [ ]:
# Add your code here

## Question 5

Visualize the `BLUE` subway stations and the streets within 200 meters of the `BLUE` subway stations on the same map using leafmap and lonboard.

In [ ]:
# Add your code here